In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import math
import torch
import pickle
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error
import joblib

import holidays
us_holidays = holidays.country_holidays('US')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

directory = '/content/drive/MyDrive/Thesis'
data_dir = directory + "/Data"
models_dir = directory + "/models"

In [3]:
from hyperopt.pyll.base import scope
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [4]:
returns = pd.read_csv(f"{data_dir}/pre_processed_data_returns.csv")

#tss = TimeSeriesSplit(n_splits=5, test_size=24*60, gap=24)
df = returns.sort_values(by=['ended_at_year', 'ended_at_month',
                            'ended_at_day', 'ended_at_hour'])
df = df.drop(columns=['Unnamed: 0', 'duration_sec']).reset_index(drop=True)
df['ended_at_daily'] = pd.to_datetime(df['ended_at_daily'])
df['ended_at_hourly'] = pd.to_datetime(df['ended_at_hourly'])

df = df[df['clusters']==1]
df = df[df['ended_at_hourly'] >= pd.to_datetime("2022-01-01 00:00:00")].drop_duplicates(
    ['ended_at_hourly', 'end_station_cluster', 'bike_return']).reset_index(drop=True)

In [5]:
earliest_trips = df.groupby('end_station_cluster').nth(0)
stations_operational_since_early = earliest_trips[(earliest_trips['ended_at_month'] < 4)]['end_station_cluster']
stations_early_enough = df['end_station_cluster'].isin(stations_operational_since_early)
print((df[~stations_early_enough].shape[0]/df.shape[0])*100)
df = df[stations_early_enough].reset_index(drop=True)

1.5319617097486418


In [6]:
CLUSTER_COLUMN = 'end_station_cluster'
DATETIME = "ended_at_hourly"
DATE = "ended_at_daily"

In [7]:
def fill_three_day_window(day, df, cluster):
    # Define the three-day window (previous day, current day, and next day)
    start_date = day - pd.Timedelta(days=1)
    end_date = day + pd.Timedelta(days=1)

    days_to_fill = []
    for i in range(3):
        candidate_day = start_date + pd.Timedelta(days=i)
        if candidate_day not in processed_dates:
            days_to_fill.append(candidate_day)
    #print(days_to_fill)
    # If all days overlap, skip filling this window
    if not days_to_fill:
        return None
    days_to_fill[-1] = days_to_fill[-1] - pd.Timedelta(hours=1)
    # Create a full hourly range only for the non-overlapping days
    full_range = pd.date_range(start=min(days_to_fill), end=max(days_to_fill), freq='h')


    # Reindex the DataFrame to this range and interpolate/forward-fill
    df_window = df.reindex(full_range)
    df_window['bike_return'] = df_window['bike_return'].fillna(0)
    #df_window['flag_added'] = df_window['flag_added'].fillna(True)
    df_window['end_station_cluster'] = cluster  # or use fillna()
    df_window['clusters'] = 1  # or use fillna()

    # Add these dates to the processed set
    new_days = [start_date + pd.Timedelta(days=i) for i in range(3)]
    new_days[-1] = new_days[-1] - pd.Timedelta(hours=1)

    processed_dates.update()

    return df_window

In [8]:
def add_lags(df, target, identifier):

  df_res = pd.DataFrame()
  print(target)
  for ii in df[identifier].unique():
      df_current = df[df[identifier]==ii].copy()
      df_current.index = df_current['ended_at_hourly']
      df_current.index = pd.to_datetime(df_current.index)
      target_map = df_current[target].to_dict()
      df_current[f"{target}_lag_1_h"] = (df_current.index - pd.Timedelta('1 hours')).map(target_map)
      df_current[f"{target}_lag_2_h"] = (df_current.index - pd.Timedelta('2 hours')).map(target_map)
      df_current[f"{target}_lag_24_h"] = (df_current.index - pd.Timedelta('24 hours')).map(target_map)
      if target == "demand":
          df_current[f"{target}_lag_1_h"] = df_current[f"{target}_lag_1_h"].fillna(0)
          df_current[f"{target}_lag_2_h"] = df_current[f"{target}_lag_2_h"].fillna(0)
          df_current[f"{target}_lag_24_h"] = df_current[f"{target}_lag_24_h"].fillna(0)
      else:
          df_current[f"{target}_lag_1_h"] = df_current[f"{target}_lag_1_h"].interpolate().fillna(0)
          df_current[f"{target}_lag_2_h"] = df_current[f"{target}_lag_2_h"].interpolate().fillna(0)
          df_current[f"{target}_lag_24_h"] = df_current[f"{target}_lag_24_h"].interpolate().fillna(0)
      df_res = pd.concat([df_res, df_current])
  return df_res

In [9]:
df_concatenated = pd.DataFrame()
for i in df[CLUSTER_COLUMN].unique():

  df_current = df[df[CLUSTER_COLUMN]==i].copy()
  #df_current['flag_added'] = False
  df_current[DATETIME] = pd.to_datetime(df_current[DATETIME])
  df_current.set_index(DATETIME, inplace=True)
  unique_dates = df_current.index.normalize().unique()

  # Create an empty DataFrame to hold the result
  df_filled = pd.DataFrame()

  # Track days that have already been processed to avoid overlap
  processed_dates = set()
  #print(i)
  # Function to generate a three-day range and fill missing hours
  # Process each unique date
  for date in unique_dates:
      df_current_date = df_current[df_current[DATE] == date].copy()
      filled_window = fill_three_day_window(date, df_current_date, i)
      #print(filled_window.iloc[:, 0:7])
      #print()
      if filled_window is not None:
          df_filled = pd.concat([df_filled, filled_window])

  # Drop duplicates (if any) and sort the DataFrame
  df_filled = df_filled[~df_filled.index.duplicated(keep='first')].sort_index()
  df_concatenated = pd.concat([df_concatenated, df_filled])

del df_filled


In [10]:
def create_datetime_features(input_df, column_name):

    input_df[column_name+'_hour'] = input_df[column_name].dt.hour
    input_df[column_name+'_quarter'] = input_df[column_name].dt.quarter
    input_df[column_name+'_month'] = input_df[column_name].dt.month
    input_df[column_name+'_year'] = input_df[column_name].dt.year
    input_df[column_name+'_week'] = input_df[column_name].dt.isocalendar().week
    input_df[column_name+'_day'] = input_df[column_name].dt.day
    input_df[column_name+'_dayofweek'] = input_df[column_name].dt.dayofweek

    return input_df

In [11]:
def add_datetime(df, column_name="started_at"):
    conversion_dict_hourly = dict(year= df[f'{column_name}_year'],
                                  month=df[f'{column_name}_month'],
                                  day=  df[f'{column_name}_day'],
                                  hour= df[f'{column_name}_hour']
                                 )
    conversion_dict_daily = dict(year= df[f'{column_name}_year'],
                           month=df[f'{column_name}_month'],
                           day=  df[f'{column_name}_day']
                                 )
    df[f'{column_name}_hourly'] = pd.to_datetime(conversion_dict_hourly)

    df[f'{column_name}_daily'] = pd.to_datetime(conversion_dict_daily)

    return df

In [12]:
df_concatenated = df_concatenated.reset_index().rename(columns={
    "index": "ended_at_hourly"
})

In [13]:
df_concatenated.rename(columns={"ended_at_hourly": "ended_at"})

,ended_at,end_station_cluster,ended_at_year,ended_at_month,ended_at_day,ended_at_hour,bike_return,ended_at_week,ended_at_quarter,ended_at_dayofweek,is_holiday,ended_at_daily,clusters
0,2021-12-31 00:00:00,102,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaT,1
1,2021-12-31 01:00:00,102,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaT,1
2,2021-12-31 02:00:00,102,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaT,1
3,2021-12-31 03:00:00,102,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaT,1
4,2021-12-31 04:00:00,102,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaT,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3045571,2023-06-16 19:00:00,210,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaT,1
3045572,2023-06-16 20:00:00,210,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaT,1
3045573,2023-06-16 21:00:00,210,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaT,1
3045574,2023-06-16 22:00:00,210,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaT,1


In [14]:
df_concatenated = add_datetime(create_datetime_features(
    df_concatenated.rename(columns={"ended_at_hourly": "ended_at"}).copy(), "ended_at"
), "ended_at")
df_concatenated = df_concatenated.drop(columns=['ended_at'])

In [15]:
df_concatenated['is_holiday'] =  df_concatenated['ended_at_hourly'].apply(lambda x: x in us_holidays)

In [16]:
weather_data = pd.read_csv(f"{data_dir}/weather_data.csv").rename(columns={"started_at_hourly": "ended_at_hourly"})
weather_data['ended_at_hourly'] = pd.to_datetime(weather_data['ended_at_hourly'])

# median_code = weather_data["coco"].median()
# weather_data["coco"] = weather_data["coco"].fillna(median_code).astype(int)

number_of_clusters = df['end_station_cluster'].nunique()

weather_data_time = weather_data.reset_index().rename(columns={
    "time": "ended_at_hourly"
})
df_concatenated = df_concatenated.merge(weather_data_time[['temp', 'dwpt', "rhum", "prcp", "wdir", "wspd", "pres", "coco", "centroid", 'ended_at_hourly']].rename(columns={
    "centroid":"clusters"
}), on=['clusters', "ended_at_hourly"], how='left')

In [17]:
df_concatenated = df_concatenated.sort_values("ended_at_hourly")
df_concatenated = add_lags(df_concatenated, "bike_return", identifier=CLUSTER_COLUMN)
df_concatenated = add_lags(df_concatenated, "temp", identifier=CLUSTER_COLUMN)
df_concatenated = add_lags(df_concatenated, "prcp", identifier=CLUSTER_COLUMN)
df_concatenated = add_lags(df_concatenated, "rhum", identifier=CLUSTER_COLUMN)
df_concatenated = add_lags(df_concatenated, "wspd", identifier=CLUSTER_COLUMN)
#df_concatenated = add_lags(df_concatenated, "coco", identifier=CLUSTER_COLUMN)

bike_return
temp
prcp
rhum
wspd


In [18]:
fold = 0
preds = pd.DataFrame()
rmse = []
mape = []
mse = []
le = preprocessing.LabelEncoder()
scaler = MinMaxScaler(feature_range=(1, 2))
SCALER_FEATURES = ["ended_at_year", "ended_at_month",
                   "ended_at_day", "ended_at_hour",
                   "ended_at_week", "ended_at_quarter",
                   "ended_at_dayofweek",
                   "temp",
                   "dwpt",
                   "rhum",
                   "prcp",
                   "wdir",
                   "wspd",
                   "pres",
                   "coco",
                   "bike_return_lag_1_h",
                   "bike_return_lag_2_h",
                   "bike_return_lag_24_h",
                   "temp_lag_1_h",
                   "temp_lag_2_h",
                   "temp_lag_24_h",
                   "prcp_lag_1_h",
                   "prcp_lag_2_h",
                   "prcp_lag_24_h",
                   "rhum_lag_1_h",
                   "rhum_lag_2_h",
                   "rhum_lag_24_h",
                   "wspd_lag_1_h",
                   "wspd_lag_2_h",
                   "wspd_lag_24_h"
                  ]
target_scaler = MinMaxScaler(feature_range=(1, 2))
df_concatenated[SCALER_FEATURES] = scaler.fit_transform(df_concatenated[SCALER_FEATURES])
df_concatenated[['bike_return_target']] = target_scaler.fit_transform(df_concatenated[['bike_return']])
df_concatenated['end_station_cluster'] = le.fit_transform(df_concatenated['end_station_cluster'])
df_concatenated['is_holiday'] = df_concatenated['is_holiday'].astype(int)
#df_concatenated['flag_added'] = df_concatenated['flag_added'].astype(int)

filename = f'{models_dir}/target_returns_scaler.sav'
joblib.dump(target_scaler, filename)

filename = f'{models_dir}/scaler_returns.sav'
joblib.dump(scaler, filename)

['/content/drive/MyDrive/Thesis/models/scaler_returns.sav']

In [19]:
df_concatenated = df_concatenated.query('~((ended_at_hour>=2) & (ended_at_hour<=5))').copy().reset_index(drop=True)

In [20]:
df_concatenated.to_csv(f'{data_dir}/final_returns_model_input.csv')